In [1]:
# # Notebook 2 — Full Retrain with Best-Model Auto-Selection

# This notebook:
# - Loads `data/processed/merged_cleaned.csv`
# - Creates `duration_hours`
# - Trains Linear Regression, Random Forest, Gradient Boosting, and (optionally) XGBoost
# - Evaluates with MAE, RMSE, R²
# - Picks the best model automatically (lowest RMSE, tie-breaker highest R²)
# - Saves the new model as `models/energy_model.pkl`
# - Exports predictions and metrics to `outputs/`


In [2]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Try XGBoost if available (optional)
try:
    from xgboost import XGBRegressor
    HAS_XGB = True
except Exception:
    HAS_XGB = False

DATA = "../data/processed/merged_cleaned.csv"
MODEL_OUT = "../models/energy_model.pkl"          # NEW model file (keeps old one intact)
PRED_OUT = "../outputs/predictions_new.csv"
METRICS_OUT = "../outputs/metrics.csv"

os.makedirs("../models", exist_ok=True)
os.makedirs("../outputs", exist_ok=True)


In [3]:
df = pd.read_csv(DATA, parse_dates=["STARTTIME_first"])
print("Loaded:", df.shape)

# Feature engineering
df["duration_hours"] = df["DURATION_SEC"] / 3600

# Final feature set (full, as agreed)
FEATURES = [
    "DURATION_SEC", "MW",
    "TEMP_mean", "TEMP_p95",
    "VALO2_mean", "VALO2_p95",
    "O2_AMOUNT_sum", "GAS_AMOUNT_sum",
    "O2_FLOW_mean", "GAS_FLOW_mean",
    "duration_hours"
]

# Keep only available features
FEATURES = [c for c in FEATURES if c in df.columns]

# Target
if "Energy_MWh" not in df.columns:
    # Fallback if not present (matches earlier logic)
    df["Energy_MWh"] = df["MW_mean"] * df["duration_hours"] if "MW_mean" in df.columns else df["MW"] * df["duration_hours"]

X = df[FEATURES].copy()
y = df["Energy_MWh"].copy()

print("X:", X.shape)
print("y:", y.shape)

# Drop rows with any NA in features or target
mask = ~X.isna().any(axis=1) & ~y.isna()
X = X.loc[mask].reset_index(drop=True)
y = y.loc[mask].reset_index(drop=True)
print("After NA drop -> X:", X.shape, "y:", y.shape)


Loaded: (20813, 13)
X: (20813, 10)
y: (20813,)
After NA drop -> X: (20813, 10) y: (20813,)


In [4]:
# Preserve chronological order
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print("Train:", X_train.shape, "Test:", X_test.shape)


Train: (16650, 10) Test: (4163, 10)


In [5]:
# Scale numeric features (helps Linear Regression; trees are fine)
preprocess = ColumnTransformer(
    transformers=[("num", StandardScaler(), FEATURES)],
    remainder="drop"
)

candidates = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(
        n_estimators=500, random_state=42, n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingRegressor(random_state=42)
}

if HAS_XGB:
    candidates["XGBoost"] = XGBRegressor(
        n_estimators=800, learning_rate=0.05, max_depth=6, subsample=0.8,
        colsample_bytree=0.8, random_state=42, tree_method="hist"
    )

candidates


{'LinearRegression': LinearRegression(),
 'RandomForest': RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=42),
 'GradientBoosting': GradientBoostingRegressor(random_state=42)}

In [6]:
results = {}

for name, est in candidates.items():
    pipe = Pipeline([("pre", preprocess), ("model", est)])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)

    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)

    results[name] = {"MAE": mae, "RMSE": rmse, "R2": r2}
    print(f"{name}: MAE={mae:.3f}  RMSE={rmse:.3f}  R2={r2:.4f}")

# Pick best by RMSE (lower is better), tie-breaker by R2 (higher is better)
res_df = pd.DataFrame(results).T
res_df_sorted = res_df.sort_values(by=["RMSE", "R2"], ascending=[True, False])
best_name = res_df_sorted.index[0]
best_name, res_df_sorted


LinearRegression: MAE=41.428  RMSE=58.575  R2=-0.4110
RandomForest: MAE=38.717  RMSE=55.252  R2=-0.2554
GradientBoosting: MAE=38.892  RMSE=55.795  R2=-0.2803


('RandomForest',
                         MAE       RMSE        R2
 RandomForest      38.716593  55.252172 -0.255446
 GradientBoosting  38.892039  55.795385 -0.280253
 LinearRegression  41.427723  58.575009 -0.410990)

In [7]:
from joblib import dump

best_model = Pipeline([("pre", preprocess), ("model", candidates[best_name])])
best_model.fit(X_train, y_train)

dump(best_model, MODEL_OUT)
print("Saved best model to:", MODEL_OUT)
print("Best model:", best_name)

# Save metrics table
res_df_sorted.to_csv(METRICS_OUT)
print("Saved metrics to:", METRICS_OUT)

# Save predictions preview for auditing
y_pred = best_model.predict(X_test)
pred_df = pd.DataFrame({
    "HEATID": df.iloc[split_idx:]["HEATID"].values if "HEATID" in df.columns else np.arange(len(y_test)),
    "Actual_MWh": y_test.values,
    "Predicted_MWh": y_pred
})
pred_df.to_csv(PRED_OUT, index=False)
print("Saved test predictions to:", PRED_OUT)

res_df_sorted


Saved best model to: ../models/energy_model.pkl
Best model: RandomForest
Saved metrics to: ../outputs/metrics.csv
Saved test predictions to: ../outputs/predictions_new.csv


,MAE,RMSE,R2
RandomForest,38.716593,55.252172,-0.255446
GradientBoosting,38.892039,55.795385,-0.280253
LinearRegression,41.427723,58.575009,-0.410990


In [8]:
df[["Energy_MWh"]].describe()


,Energy_MWh
count,20813.000000
mean,138.154316
std,54.905755
min,-2682.048600
25%,111.128472
50%,131.224653
75%,156.893939
max,1818.419234


In [9]:
df[FEATURES].describe()


,DURATION_SEC,TEMP_mean,TEMP_p95,VALO2_mean,VALO2_p95,O2_AMOUNT_sum,GAS_AMOUNT_sum,O2_FLOW_mean,GAS_FLOW_mean,duration_hours
count,20813.000000,20813.000000,20813.000000,20813.000000,20813.000000,2.081300e+04,2.081300e+04,20813.000000,20813.000000,20813.000000
mean,124368.589824,1634.906913,1653.445366,860.049840,1365.414697,5.407789e+05,1.271020e+05,2101.331345,783.391200,34.546831
std,11769.531908,16.902751,16.901203,642.696153,1753.646750,1.660051e+05,5.404551e+04,946.749876,215.341895,3.269314
min,1080.000000,1517.000000,1517.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.300000
25%,119520.000000,1623.750000,1642.200000,581.800000,786.900000,4.629810e+05,1.048780e+05,1382.925651,618.872093,33.200000
50%,124860.000000,1634.250000,1651.800000,783.000000,958.100000,5.201660e+05,1.206910e+05,1779.988506,763.120968,34.683333
75%,130200.000000,1645.333333,1662.800000,955.857143,1154.800000,5.930030e+05,1.403150e+05,2549.635514,908.466926,36.166667
max,310200.000000,1737.000000,1818.200000,8141.600000,9999.000000,1.536803e+07,5.564226e+06,5399.058594,1783.043860,86.166667


In [10]:
df.nunique()


HEATID             20813
TAP_first             17
STARTTIME_first    20813
DURATION_SEC        1079
MW_mean             2031
TEMP_mean           1962
TEMP_p95            1781
VALO2_mean          9517
VALO2_p95          12027
O2_AMOUNT_sum      20181
GAS_AMOUNT_sum     18768
O2_FLOW_mean       20804
GAS_FLOW_mean      20794
duration_hours      1079
Energy_MWh         18260
dtype: int64

In [11]:
print(len(X_train), len(X_test))


16650 4163


In [12]:
df.head(10)


,HEATID,TAP_first,STARTTIME_first,DURATION_SEC,MW_mean,TEMP_mean,TEMP_p95,VALO2_mean,VALO2_p95,O2_AMOUNT_sum,GAS_AMOUNT_sum,O2_FLOW_mean,GAS_FLOW_mean,duration_hours,Energy_MWh
0,5F0002,14,2015-01-01 01:12:00,60600.0,6.078125,1627.000000,1648.3,383.000000,400.70,428122.0,175048.0,3963.962264,1527.245283,16.833333,102.315104
1,5F0003,11,2015-01-01 01:41:00,120600.0,6.078125,1641.000000,1651.8,683.000000,696.50,382714.0,156025.0,3771.399194,1556.326613,33.500000,203.617188
2,5F0004,11,2015-01-01 02:26:00,123600.0,3.644231,1636.000000,1636.0,700.000000,700.00,606453.0,251531.0,3139.359882,1206.572271,34.333333,125.118590
3,5F0005,11,2015-01-01 03:27:00,128460.0,4.477273,1630.000000,1639.7,625.333333,663.60,453178.0,179407.0,4009.954887,1532.003759,35.683333,159.764015
4,5F0006,11,2015-01-01 04:16:00,126000.0,4.534091,1622.000000,1644.8,654.600000,756.00,460713.0,178593.0,3881.255556,1504.070370,35.000000,158.693182
5,5F0007,11,2015-01-01 06:00:00,120000.0,5.781250,1647.000000,1648.8,601.000000,630.70,433331.0,165114.0,3957.713768,1422.884058,33.333333,192.708333
6,5F0008,11,2015-01-01 06:56:00,123000.0,5.444444,1607.250000,1639.0,544.500000,653.60,532232.0,195028.0,4299.223729,1416.667797,34.166667,186.018519
7,5F0009,11,2015-01-01 07:48:00,117000.0,5.765625,1653.000000,1653.9,698.500000,736.75,377021.0,130931.0,3443.829876,1219.307054,32.500000,187.382812
8,5F0011,11,2015-01-01 09:22:00,134340.0,3.325000,1649.000000,1673.8,612.750000,716.35,498855.0,170152.0,2966.620690,1048.289655,37.316667,124.077917
9,5F0013,11,2015-01-01 11:14:00,128400.0,6.250000,1643.333333,1665.7,608.666667,691.90,414849.0,136868.0,4049.921260,1161.259843,35.666667,222.916667


In [13]:
from joblib import load
model = load("../models/energy_model.pkl")

sample = X_test.iloc[:5]
preds = model.predict(sample)
preds


array([149.10304955, 135.57153529, 137.2993493 , 159.83122803,
       139.72299793])

In [14]:
model


,steps,"[('pre', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
